In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

---

## Økonometri A

# Problem set 5

---



### Vækstregressioner: Absolut og betinget konvergens

I denne ugeseddel skal I udvide regressionsmodellen fra ugeseddel 3, hvor I så på hypotesen om absolut og betinget konvergens, ved at inkludere information om hvert lands humankapital. I Makroøkonomi A vil I se, at en model uden humankapital vil underestimere betydningen af opsparing og befolkningsvækst på BNP pr. arbejder og samtidig overestimere hastigheden af, hvor hurtigt et land konvergerer mod en vækstbane.

I skal tilføje en variabel i regressionsmodellen, som viser investeringsraten i humankapital korrigeret for befolkningsudviklingen, afskrivningsrate og teknologisk fremgang. I skal ligesom i ugeseddel 3 anvende en afskrivningsrate og vækst i teknologi på 0,075. Som en udvidelse til ugeseddel 3 skal I formelt teste forskellige hypoteser om absolut og betinget konvergens. I skal forklare, hvornår I skal bruge et _F_-test og et _t_-test og under hvilke antagelser, de giver valide resultater.

Regressionsmodellen med humankapital kan omskrives til:

$$
gy_i = \beta_0 + \beta_1 \log (y60_i) + \beta_2 strucK_i + \beta_3 strucH_i + \epsilon_i \tag{1}
$$

hvor de strukturelle karakteristika er 
- $ strucK_i = \ln(sk_i) - \ln(n_i + 0,075) $ 
- og $  strucH_i = \ln(sh_i) - \ln(n_i + 0,075) $

I skal anvende det samme datasæt, som I brugte til ugeseddel 3.



---

### Gruppespørgsmål


#### Opgave 1 
I kan pålægge en model parameterrestriktioner for at omskrive den til en mere simpel model og indføre disse som en nulhypotese. Den udvidede Solow-model (1) indeholder også den simple regressionsmodel fra spørgsmål 4 i ugeseddel 3, hvor I så på hypotesen om absolut konvergens. Hvad er parameterrestriktionerne og derved nulhypotesen i den udvidede model (1), hvis I gerne vil teste hypotesen om absolut konvergens? Hvad er alternativ hypotesen?



**Dit svar:**

> Parameterrestiktionerne i den udvidede Solow model (1) for at teste hypotesen om absolut konvergens er, at de strukturelle karakteristika ikke har en betydning i modellen. Nulhypotesen er $H_0: \beta_2 = \beta_3 = 0$. Alternativ hypotesen er $H_A: \beta_2 \neq 0 \vee \beta_3 \neq 0$.

#### Opgave 2 
Giv et forslag til et relevant test, som I kan bruge til at teste nulhypotesen i gruppespørgsmål 1. Hvad er antallet af parameterrestriktioner, som I gerne vil teste?



**Dit svar:**

> For at teste nulhypotesen i gruppespørgsmål 1 skal vi bruge et _F_-test, da der er to parameterrestriktioner.

#### Opgave 3

Hvilke antagelser skal være opfyldt for, at det foreslåede test i gruppespørgsmål 2 er validt?



**Dit svar:**

> Antagelserne MLR.1 til MLR.6 skal være opfyldt for, at _F_-testet er validt (slå dem op, hvis du har behov for det). Antagelsen om normalfordelte fejlled (MLR.6) er en restriktiv antagelse – den kan erstattes af en antagelse om, at fejlledet går mod at være normalfordelt, når antallet af observationer går imod uendelig.  

---

### Python øvelser



#### Opgave 1
Indlæs data i Python. Fjern de lande, som har datakvalitet D, og lande, som mangler observationer for investeringsraten i humankapital, **sh**. Herefter bør I have 77 lande tilbage.

_Hint:_ Hvis du vil tjekke om en variabel indeholder manglende observationer kan du bruge en af de to komplementære kommandoer:
```py
df['sh'].notnull()
df['sh'].isnull()

```

**Din kode:**

In [2]:
df = pd.read_stata('PS3.dta')
df = df[df.grade != 'D']
df = df[df.sh.notnull()]
df

,country,iso3,y03,y60,gy,sk,sh,u,isi,n,grade
1,USA,USA,1.000000,1.000000,0.017709,0.189912,0.115346,12.049,0.97333,0.016080,A
2,Ireland,IRL,0.971402,0.423321,0.037025,0.209641,0.125219,9.351,0.76672,0.010019,A
3,Norway,NOR,0.968075,0.761374,0.023295,0.283681,0.103698,11.848,0.87272,0.012205,A
4,Belgium,BEL,0.906815,0.663650,0.024969,0.224658,0.101057,9.338,0.86572,0.004503,A
5,Austria,AUT,0.880988,0.554981,0.028456,0.240480,0.095199,8.354,0.86356,0.002440,A
...,...,...,...,...,...,...,...,...,...,...,...
91,Tanzania,TZA,0.025963,0.028716,0.015365,0.048912,0.006915,2.705,0.27567,0.028053,C
92,Malawi,MWI,0.023686,0.026768,0.014864,0.096660,0.011803,3.204,0.25133,0.025739,C
93,Madagascar,MDG,0.023345,0.078392,-0.010462,0.039704,0.030596,NaN,0.23800,0.025138,C
94,Ethiopia,ETH,0.023061,0.026356,0.014604,0.035875,0.014008,9.988,0.19933,0.021697,C


---
#### --- INTERMISSION --- 
Inden vi går videre med de næste opgaver, skal I lære en smart genvej i `statsmodels`, som gør det lidt simplere at køre regressioner. 

Indtil videre har vi manuelt udformet vores $X$-matrix, tilføjet en konstant og udformet vores $y$-vektor. Alt det kan vi faktisk springe over og i stedet bare specificere vores ønskede model med en tekststreng. Til det formål skal vi lige importere et nyt model fra statsmodels:

```py
import statsmodels.formula.api as smf
```

Prøv nu fx at regressere BNP vækst (`gy`) på befolkningstilvækst (`n`) og investeringsraten i kapital per indbygger (`sk`). Brug kommandoen
```py
results = smf.ols('gy ~ n + sk', data=df).fit()
print(results.summary())
```

Som du kan se af outputtet sørger statsmodels selv for at tilføje en konstant. Hvis du vil tilføje flere forklarende variable til regressionen, skal du bare tilføje flere led til tekststrengen


**Din kode:** 

In [3]:
import statsmodels.formula.api as smf

results = smf.ols('gy ~ n + sk', data=df).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     gy   R-squared:                       0.277
Model:                            OLS   Adj. R-squared:                  0.258
Method:                 Least Squares   F-statistic:                     14.18
Date:                Thu, 12 Sep 2024   Prob (F-statistic):           6.11e-06
Time:                        20:19:09   Log-Likelihood:                 242.27
No. Observations:                  77   AIC:                            -478.5
Df Residuals:                      74   BIC:                            -471.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0118      0.005      2.466      0.0

--- 

#### Opgave 2
Estimer den udvidede model (1) med `statsmodels`. Hvad er fortolkningen af $ \widehat{\beta}_1 $? Sammenlign estimatet af $ \beta_1 $ fra spørgsmål 7 i ugeseddel 3.



**Din kode:**

In [4]:
df['lny60'] = np.log(df.y60)
df['strucK'] = np.log(df.sk) - np.log(df.n + 0.075)
df['strucH'] = np.log(df.sh) - np.log(df.n + 0.075)

results = smf.ols('gy ~ lny60 + strucK + strucH', data=df).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     gy   R-squared:                       0.418
Model:                            OLS   Adj. R-squared:                  0.394
Method:                 Least Squares   F-statistic:                     17.50
Date:                Thu, 12 Sep 2024   Prob (F-statistic):           1.17e-08
Time:                        20:19:09   Log-Likelihood:                 250.63
No. Observations:                  77   AIC:                            -493.3
Df Residuals:                      73   BIC:                            -483.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0029      0.003      0.998      0.3

**Dit svar:**

> Estimation af den udvidede Solow model (1). Fortolkningen af $\widehat{\beta}_1$ er, at hvis BNP pr. arbejder er 1 pct. større i 1960, vil den årlige vækstrate i BNP pr. arbejder fra 1960 til 2003 være -0,0075 pct. point lavere. 
>
> Estimatet af $\beta_1$ i den udvidede Solow model er numerisk større end i modellen fra ugeseddel 3. Det kan forklares ud fra den indledende tekst i ugesedlen, at en model uden human kapital vil overestimere konvergenshastigheden.

#### Opgave 3
Hvad er nul- og alternativ hypotesen for modellens samlede signifikans? Hvilket test skal I bruge for at teste nulhypotesen? Hvad er teststørrelsen og den kritiske værdi, og kan nulhypotesen afvises? Vær præcise, når I formulerer nul- og alternativ hypotesen i ord.

_Hint:_ for at finde den kritiske værdi til et _F_-test kan du bruge Python-pakken scipy:
```py
import scipy
q = ... # det ønskede signifikansniveau
dfn = ... # antal frihedsgrader i tælleren
dfd = ... # antal frihedsgrader i nævneren
scipy.stats.f.ppf(1-q, dfn, dfd)
```

In [5]:
import scipy
q = 0.05
dfn = 3
dfd = 73
scipy.stats.f.ppf(1-q, dfn, dfd)

2.7300187139961727

**Dit svar:**

> Nulhypotesen for modellens samlede signifikans er $H_0: \beta_1 = \beta_2 = \beta_3 = 0$. Alternativ hypotesen er $H_A:$ mindst én $\beta_j \neq 0, j=1,2,3$. 
> 
> Hvis vi skal formulere nulhypotesen i ord, er den, at BNP pr. arbejder i 1960 og strukturelle karakteristika ikke samlet set kan forklare væksten i BNP pr. arbejder fra 1960 til 2003, hvormed hypotesen om absolut og betinget konvergens ikke er understøttet empirisk.
>
> Vi skal anvende et _F_-test for at teste nulhypotesen. Aflæs _F_-teststørrelsen i regressionsoutputtet i Python til $F(3,73)=17,50$. Den kritiske værdi på et 5 pct. signifikansniveau er 2,73. Når den kritiske værdi er mindre end teststørrelsen, kan nulhypotesen afvises, så BNP pr. arbejder i 1960 og strukturelle karakteristika samlet set kan forklare væksten i BNP pr. arbejder.


#### Opgave 4
Hvad er nul- og alternativ hypoteserne for hver af de individuelle forklarende variable? Hvilket test skal I bruge for at teste nulhypoteserne? Hvad er teststørrelserne og den kritiske værdi, og kan nulhypoterne afvises? Vær præcise, når I formulerer nul- og alternativ hypoteserne i ord.


**Dit svar:**

In [6]:
scipy.stats.t.ppf(1-0.025, df=77)

1.9912543951146038

> Nulhypoteserne for hver af de individuelle forklarende variable er $H_0: \beta_j = 0, j=1,2,3$. Alternativ hypotesen er $H_A: \beta_j \neq 0, j=1,2,3$. Hvis vi skal formulere en af nulhypoteserne i ord, f.eks. for $\beta_2$, er det, at fysisk kapital ikke kan forklare væksten i BNP pr. arbejder fra 1960 til 2003. 
> 
> Vi skal anvende _t_-test til at teste de tre nulhypoteser for hver af de forklarende variable. Aflæs _t_-teststørrelserne i regressionsoutputtet i Python til hhv. -4,44, 4,61 og 1,98. Den kritiske værdi på et 5 pct. signifikansniveau kan findes enten ud fra en standard normalfordeling til $\pm$ 1,96 eller ud fra en _t_-fordeling. Her kan vi anvende scipy med kommandoen `scipy.stats.t.ppf(q=1-0.025, df=77)` og finde den kritiske værdi til 1,99. Den kritiske værdi er ikke helt ens med de to metoder, da antallet af observationer i lavt. Estimaterne af $\beta_1$ og $\beta_2$ er signifikante, så både BNP pr. arbejder i 1960 og fysisk kapital kan forklare væksten i BNP pr. arbejder. Estimatet af $\beta_3$ er borderline, da _t_-teststørrelsen er tæt på den kritiske værdi.  


#### Opgave 5
Hvad er en passende alternativ hypotese, når I tester $ \beta_1 = 0 $ i model (1)? Udfør testet. 

[Hint: Hvilken betydning har hypotesen om betinget konvergens for $ \beta_1 $? Tag udgangspunkt i gruppespørgsmål i ugeseddel 3]


In [7]:
scipy.stats.t.ppf(1-0.05, df=77)

1.6648845371855607

> En passende alternativ hypotese, når vi skal teste hypotesen om betinget konvergens, er $H_A: \beta_1 < 0$, da initialt fattige lande skal have en højere vækst i BNP pr. arbejder, end lande, som er rige i 1960. _t_-teststørrelsen er den samme som i spørgsmål 4. Den kritiske værdi skal afspejle et en-sidet test, som kan findes enten ud fra en standard normalfordeling til -1,64 eller ud fra en _t_-fordeling. Her kan vi anvende Python kommandoen `scipy.stats.t.ppf(q=0.95, df=77)` og finde den kritiske værdi til -1,66. Nulhypotesen $\beta_1=0$ kan afvises, når teststørrelsen er numerisk mindre end den kritiske værdi, så hypotesen om betinget konvergens bliver understøttet empirisk i dette tilfælde.


#### Opgave 6
I skal nu undersøge, om de strukturelle karakteristika samlet set kan udelades fra regressionsmodellen. Opskriv den relevante nulhypotese til F-testet ved at tage udgangspunkt i gruppespørgsmålene.

1. Udregn _F_-testet i hånden. I skal tage udgangspunkt i ligning [4.37] i Wooldridge (7. udgave). I skal bruge resultaterne fra spørgsmål 2 samt estimere en restrikteret model, hvor nulhypotesen fra gruppespørgsmålene er pålagt.

2. Lad Python udregne _F_-testet. Hvis dit OLS resultat-objekt fra statsmodels hedder `results` kan du bruge følgende kommando:

    ```py
    hypotheses = '(strucK = strucH = 0)'
    f_test = results.f_test(hypotheses)
    print(f_test)
    ```
3. Tjek at resultaterne  er ens.


**Din kode:**

In [24]:
# Manual calculation of F_test
## 1. Get the SSR of the restricted model
results_r = smf.ols('gy ~ lny60', data=df).fit()
SSR_r = results_r.ssr

## 2. Get the SSR of the unrestricted model
results_ur = smf.ols('gy ~ lny60 + strucK + strucH', data=df).fit()
SSR_ur = results.ssr

## 3. Calculate the F-test
F_test_manual = ((SSR_r-SSR_ur)/2) / (SSR_ur/(77-3-1))
print('F-test')
print(F_test_manual)

## 4. Print critical value
print('Critical value')
print(scipy.stats.f.ppf(1-0.05, 2, 73))

F-test
26.233555200023
Critical value
3.1221029319882803


In [22]:
# Statsmodels F-test
hypotheses = '(strucK = strucH = 0)'
f_test = results.f_test(hypotheses)
print(f_test)


<F test: F=26.233555200022995, p=2.5998194642764728e-09, df_denom=73, df_num=2>


**Dit svar:**

>
> 1.  Ligning [4.37]: 
> \begin{align*}
>     F = \frac{SSR_r-SSR_{ur}/q}{SSR_{ur}/(n-k-1)}
> \end{align*}
> Vi skal finde residual variationen fra hhv. den restrikterede og urestrikterede model. $q$ er antallet af parameterrestriktioner, $n$ er antallet af observationer og $k$ er antallet af parametre i modellen. _F_-teststørrelsen bliver 26,23. <br><br>
> Vi finder den kritiske værdi med kommandoen 
> `scipy.stats.f.ppf(0.95, 2, 73)`
>  til 3,12. Nulhypotesen om, at de strukturelle karakteristika ikke har en betydning for modellen, og at modellen kan reduceres til den simple model med absolut konvergens, kan afvises. Dette giver empirisk evidens for betinget konvergens. 
> 
> 2. _F_-teststørrelsen er den samme, som vi fandt med formlen: $F(2,73) =$ 26,23. Ligeledes med den kritiske værdi.
> 
> 3. Resultaterne er ens med de to metoder.
>


#### Opgave 7
I dette spørgsmål skal I implementere jeres egen version af OLS-estimatoren i MLR-tilfældet – ligesom i Problem Set 3. Denne gang skal jeres implementerede funktion dog både returnere OLS-parameterestimaterne og deres standardfejl.
 
Ligning (2) og (3) nedenfor viser formlen for OLS-estimatoren med homoskedastiske standardfejl.

$$
\widehat{\beta} = (X'X)^{-1}X'y \tag{2}
$$

$$
\text{var}(\widehat{\beta} | X) = \sigma^2 (X'X)^{-1}, \quad \sigma^2 = \frac{1}{N-K} \widehat{U}' \widehat{U} \tag{3}
$$

hvor 
- $ N $ er antallet af observationer, 
- $ K $ er antallet af parametre inkl. et konstantled 
- $ \widehat{U} \equiv y - X\widehat{\beta} $ er residualerne.
- $ \sigma^2 $ er residualvariansen
- Standardfejlene er kvadratroden af diagonalelementerne i kovarians-matricen $\text{var}(\widehat{\beta} | X)$

Brug din funktion til at estimere model (1) og bekræft, at du får de samme parameterestimator og standardfejl som statsmodels giver.

_Tip:_ Du kan tilgå diagonalelementerne af en matrix $A$ i numpy ved hjælp af `np.diagonal(A)`

In [78]:
def ols(X, y):
    beta_hat = np.linalg.inv(X.T @ X) @ X.T @ y
    U_hat = y - X @ beta_hat
    N = X.shape[0]
    K = X.shape[1]
    SSR = U_hat.T @ U_hat
    sigma2 = SSR * 1/(N - K)
    var_beta_hat = sigma2 * np.linalg.inv(X.T @ X)
    se_ols = np.sqrt(np.diag(var_beta_hat))
    
    return beta_hat, se_ols    


Kør nedenstående celle for at estimere model (1) ved hjælp af din egen funktion:

In [92]:
X = df[['lny60', 'strucK', 'strucH']].values
X = sm.add_constant(X)
y = df['gy']

beta_hat, se_beta_hat = ols(X, y)

print('vars')
print('const, lny60, strucK, strucH')
print('beta_hat')
print(beta_hat.round(4))
print('se_beta_hat')
print(se_beta_hat.round(4))

vars
const, lny60, strucK, strucH
beta_hat
[ 0.0029 -0.0075  0.0124  0.0054]
se_beta_hat
[0.0029 0.0017 0.0027 0.0027]
